## Analyzing localization of gravitational-wave sources from GWTC

In [ ]:
from typing import Optional

import ipydatagrid
import arviz as az
from astropy.coordinates import SkyCoord
from astropy.table import Table, QTable
import astropy.units as u
import h5py
import healpy as hp
import numpy as np
from numpy.typing import NDArray, ArrayLike
from ligo.skymap.postprocess.ellipse import find_ellipse
import ligo.skymap.plot
import matplotlib.pyplot as plt

from gwtc.source import approximant_to_label, label_to_approximant

%config InlineBackend.figure_format = "retina"

In [ ]:
path_to_localization_samples = "../parsed_samples.hdf5"
path_to_output_file_prefix = "../gwtc-localization-samples-best-localized"
approximants = ["IMRPhenomXPHM", "SEOBNRv4PHM"]
analyses = approximants + ["Mixed"]
output_analysis = "Mixed"
path_to_output_file = f"{path_to_output_file_prefix}-analysis={output_analysis}.hdf5"

In [ ]:
samples_file = h5py.File(path_to_localization_samples, mode="r")
events = list(samples_file.keys())
event_table = Table({"Events": events})
event_table

## Per-event posterior samples

### Binary black holes

The [GWTC catalogue data releases](https://gwosc.org/eventapi/html/GWTC/) contain posterior samples from parameter estimation using two waveform approximants, IMPRhenomXPHM and SEOBNRv4PHM. The number of posterior samples for each event is different between the approximants, most of them coming from IMRPhenomXPHM. There is also a `Mixed` sample, which combines the results from both pipelines.

In the cell below, we display the total number of samples per event, as well as the fraction of samples in each analysis.

In [ ]:
events = list(samples_file.keys())
data = []
for event, group in samples_file.items():
    row = {label_to_approximant(label): samples.attrs["nsamples"] for label, samples in group.items()}
    total_samples = sum(row.values())
    data.append({
        "Total samples": total_samples, 
        **{k: v / total_samples for k,v in row.items()}
    })

# Reverse list of dicts into dict of lists
cols = list(data[0].keys())
data = {col: [row.get(col, np.nan) for row in data] for col in cols}

for colname, col in data.items():
    event_table[colname] = col

def format_columns(table, float_fmt="1.2f", int_fmt="1.1e"):
    for column in table.colnames:
        if table[column].dtype == np.int64 and int_fmt is not None:
            table[column].info.format = int_fmt
        if table[column].dtype == np.float64 and float_fmt is not None:
            table[column].info.format = float_fmt

def get_renderer_by_format(fmt):
    return ipydatagrid.TextRenderer(
        text_value=ipydatagrid.VegaExpr(f"format(cell.value, '{fmt}')")
    )

def pretty_show_in_notebook(table, float_fmt="1.2f", int_fmt="1.1e"):
    """
    Use astropy Table's `show_in_notebook` method to display a dynamic table
    on a notebook, and customize the formatting of floats and ints.
    """
    renderers = {}
    float_renderer = get_renderer_by_format(float_fmt)
    int_renderer = get_renderer_by_format(int_fmt)
    for col in table.colnames:
        if table[col].dtype == np.float64 and float_fmt is not None:
            renderers[col] = float_renderer
        elif table[col].dtype == np.int64 and int_fmt is not None:
            renderers[col] = int_renderer
    return table.show_in_notebook(renderers=renderers, auto_fit_columns=True)
            

# Pretty printing numbers
pretty_show_in_notebook(event_table)

## Analyzing sky localization of all events in the catalog

In the following cells, we select all events in GWTC, distingushing those which were detected by at least three detectors (LIGO Hanford, LIGO Livingston and Virgo) from others which were detected while only two interferometers were operating.

We expect the source localization to be more precise for the former set of events. To visualize this, we construct a table of the 90% credible localization area $\Delta \Omega_{90\%}$ for each event and for both waveform approximants. The area has been calculated by the LVK collaboration and is included in the data products.

In [ ]:
event_detectors = {}

# Find out which interferometers detected each event
for event, group in samples_file.items():
    detectors = []
    # The detectors are stored in a funny way, in that they are sometimes recorded in only one analysis,
    # despite being the same for all of them in practice. Because of this, we loop over everything
    label_with_detectors = None
    for label, approximant_data in group.items():
        detectors_for_label = approximant_data.attrs["detectors"]
        if all([det is not None for det in detectors_for_label]) and len(detectors_for_label) > len(detectors):
            detectors = detectors_for_label
            label_with_detectors = label
            
    event_detectors[event] = {"label": label_with_detectors, "detectors": detectors}

three_detector_events = {k for k,v in event_detectors.items() if len(v["detectors"]) >= 3}
print(f"Number of events detected by L1 H1 V1: {len(three_detector_events)}")

def get_confidence_area_label(analysis: str):
    return f"[{analysis}] Confidence area (90%)"

def get_skymap_stats_for_event(event_group: h5py.Group, analysis: str) -> dict:
    try:
        approximant_data = event_group[approximant_to_label(analysis)]
        area = approximant_data.attrs["area90"]
    except KeyError:
        area = np.nan
    return {get_confidence_area_label(analysis): area * u.degree ** 2}
    

def get_snr_stats_for_event(event: str, event_group: h5py.Group) -> dict:
    label = event_detectors[event]["label"]
    approximant_data = event_group[label]
    for detector in ["H1", "L1", "V1"]:
        try:
            snr = approximant_data[f"{detector}_matched_filter_abs_snr"][()]
            median_snr = np.median(snr)
        except KeyError:
            median_snr = np.nan
        data[f"{detector} median SNR"] = median_snr
    return data
        

data_list = []
for event, group in samples_file.items():
    data = {"Event": event, "Detected by": event_detectors[event]["detectors"]}
    for approximant in approximants:
        skymap_stats = get_skymap_stats_for_event(group, approximant)
        data.update(skymap_stats)

    snr_stats = get_snr_stats_for_event(event, group)
    data.update(snr_stats)
    data_list.append(data)

confidence_area_labels = list(map(get_confidence_area_label, approximants))
event_localization_table = QTable(rows=data_list)
event_localization_table["Detected by"].format = lambda x: " ".join(x)
format_columns(event_localization_table, float_fmt=".1f", int_fmt=".1e")
event_localization_table.sort(confidence_area_labels)
pretty_show_in_notebook(event_localization_table)

Visualizing the credible localization areas in a histogram:

In [ ]:
fig, ax = plt.subplots()
nbins = 10
plot_labels = []
three_detector_mask = np.asarray([len(event_det) >= 3 for event_det in event_localization_table["Detected by"]])
for mask, mask_label in [(three_detector_mask, "three detectors"), (~three_detector_mask, "two detectors")]:
    for approximant, label in zip(approximants, confidence_area_labels):
        data = event_localization_table[label]
        nan_mask = np.isnan(data)
        mask = np.logical_and(~nan_mask, mask)
        valid_data = data[mask]
        bins = np.geomspace(valid_data.min(), valid_data.max(), nbins)
        plot_label = f"{approximant}, {mask_label}"
        ax.hist(valid_data, bins=bins, label=plot_label, histtype="stepfilled", alpha=0.8)
        plot_labels.append(plot_label)

ax.set(xlabel="Confidence area (90%) [deg2]", ylabel="Number of events", xscale="log")
ax.grid()
_ = fig.legend(plot_labels, loc="outside upper right", ncols=len(approximants))

The last two events, GW200308_173609 and GW200322_091133, seem to be very poorly localized, with $\Delta \Omega_{90\%}$ almost spanning the whole sky. Even though they were detected when the whole network as operating, the network SNR for each of them is less than 5.



Let us take a closer look at their posterior samples:

In [ ]:
outlier_events = ["GW200308_173609", "GW200322_091133"]
nrows = len(outlier_events)
npts = 100
#fig, axs = plt.subplots(2 * nrows, 2 * 2, figsize=(16 * nrows, 6 * 4))
coords = {"ra": np.linspace(-0.5 * np.pi, 0.5 * np.pi, npts), "dec": np.linspace(0, 2 * np.pi)}
for i, event in enumerate(outlier_events):
    event_group = samples_file[event]
    for j, approximant in enumerate(approximants):
        label = approximant_to_label(approximant)
        ra, dec = event_group[label]["ra"], event_group[label]["dec"][()]
        ds = az.convert_to_inference_data({"ra": ra, "dec": dec}, coords=coords)
        #axs_to_plot = axs[2*i:2*i+2, 2*j:2*j+2]
        axs = az.plot_pair(ds, marginals=True)
        fig = axs.flatten()[0].get_figure()
        fig.suptitle(f"{event} - {approximant}")

In [ ]:
def counts_from_point_sources(
    theta: NDArray,
    phi: NDArray,
    nside: int,
    normalize: bool = False,
    weights: Optional[ArrayLike] = None,
    **kwargs,
):
    indices = hp.ang2pix(nside, theta, phi, **kwargs)
    npix = hp.nside2npix(nside)
    counts = np.bincount(indices, weights=weights, minlength=npix).astype(np.float64)
    if not normalize:
        return counts
    pix_sum = np.sum(counts)
    return counts / pix_sum if pix_sum > 0 else counts

In [ ]:
event = "GW200308_173609"
nside = 32
area_per_pixel = hp.nside2pixarea(nside, degrees=True)
fig = plt.figure(figsize=(8, 6))
data = samples_file[event][label]
ra, dec = data["ra"][()], data["dec"][()]

points = np.column_stack((ra, dec))
mean_ra, mean_dec = np.mean(ra), np.mean(dec)
center = SkyCoord(frame="icrs", ra=mean_ra, dec=mean_dec, unit="rad")
ax = fig.add_subplot(projection="astro degrees mollweide")
theta, phi = 0.5 * np.pi - dec, ra
counts = counts_from_point_sources(theta, phi, nside, normalize=True)
ax.imshow_hpx(counts, cmap="cylon")
ax.set_title(event)
ax.grid()

We estimate the sky localization uncertainty in two ways. First, we use the [`find_ellipse`](https://lscsoft.docs.ligo.org/ligo.skymap/postprocess/ellipse.html) method implemented in the `ligo.skymap` package. The method finds the ellipse containing a given probability, returning its area among other properties.

We also compare this calculation with the (naive) greedy estimate of ranking HEALPix pixels by their counts and returning the smallest number that contains the desired fraction of samples.

In [ ]:
level = 90
level_dec = level / 100
projection = "MOL"
ellipse = find_ellipse(counts, cl=level, projection=projection)
n_pix = np.searchsorted(np.cumsum(np.sort(counts)[::-1]), level_dec)
sky_localization_uncertainty = n_pix * area_per_pixel
print(f"Area at {level}% [Ellipse]: {ellipse[-1]:.2f} deg^2")
print(f"Area at {level}% [HEALPix]: {sky_localization_uncertainty:.2f} deg^2")

## Distance distributions

In the cell below, we compare the distance posteriors between analyses for each event. For the same outlier events that we presented above, the posteriors seem to be bounded by the prior. Therefore, we will remove them from the output file.

In [ ]:
fig, axs = plt.subplots(len(events), 1, figsize=(8, 1.5 * len(events)))
dl_min = np.inf
dl_max = -np.inf
legend_set = False
colors = ["tab:orange", "tab:blue", "tab:green"]
for event, ax in zip(samples_file.keys(), axs):
    event_group = samples_file[event]
    handles = []
    for approximant, color in zip(analyses, colors):
        try:
            label = approximant_to_label(approximant)
            samples = event_group[label]["luminosity_distance"][()]
            dl_min = min(dl_min, samples.min())
            dl_max = max(dl_max, samples.max())
            grid, pdf = az.kde(samples)
            line, = ax.semilogx(grid, pdf, label=approximant, color=color)
            handles.append(line)
        except KeyError:
            continue
    if len(handles) == len(analyses) and not legend_set:
        plt.figlegend(handles=handles, loc="upper right")
        legend_set = True
    ax.grid()
    ax.set(title=event, xlabel=r"$D_L \, [\rm{Mpc}]$", ylabel="PDF")
for ax in axs:
    ax.set_xlim([dl_min, dl_max])
fig.tight_layout()

Storing all localization samples for the corresponding `analysis` in a separate file, skipping outlier events:

In [ ]:
output_label = approximant_to_label(output_analysis)
with h5py.File(path_to_output_file, mode="w") as output_file:
    for event, *confidence_areas in event_localization_table.iterrows("Event", *confidence_area_labels):
        if event in outlier_events:
            continue
        try:
            group = samples_file[event]
            group.copy(group[output_label], output_file, event, without_attrs=True)
            confidence_areas = np.asarray([area.value for area in confidence_areas])
            confidence_area_90 = confidence_areas[np.isfinite(confidence_areas)][-1]
            output_file[event].attrs["confidence_area_90"] = confidence_area_90
        except KeyError:
            print(f"Analysis {output_analysis} not found for event {event}") 
    